## This script demonstrates querying all records using specified criteria through RePorter API.

In [1]:
#import the packages needed to run this script
import requests
import json

### Retrieving All Results via Iteration

The RePorter API functions like a waiter or librarian, receiving requests from you with criteria specified in a Python dictionary. The available request parameters are listed in the RePorter API documentation (https://api.reporter.nih.gov/?urls.primaryName=V2.0). After reciving a request, The API then delivers records that match the criteria.

However, there is a limitation: the RePorter API has a return limit of 500 records per request. If there are 1000 records in total, to retrieve record #501, the "offset" parameter must be set to 500. To retrieve all matching records, multiple requests must be made, incrementing the offset value each time. This process is performed in a loop in the script, and is demonstrated in the following code blocks.

## Demo: All projects of NCI in 2021

### Option 1: Using a for loop

Let's determine the number of iterations we need to retrieve all matching results using a for loop. We start by finding out the total number of records that match our search criteria, which are all NCI projects in 2021. To do this, we make an initial request with the specified criteria. The returned results are saved in a JSON format, with the total number of all records stored in its metadata under "total". Remember, the criteria are specified in the "data" 

In [2]:
# Specifying the target URL and header
url = "https://api.reporter.nih.gov/v2/projects/search"
headers = {'accept': 'application/json', 'Content-Type': 'application/json'}


#The "data" dictionary contains the criteria for the API request. 
#In this case, it is set to retrieve all projects from NCI in the year 2021.
data = {
         "criteria":
         {
             "fiscal_years":[2021],
             "agencies":["NCI"],
         #In addition to the criteria already specified, other parameters can be included in the request 
         #such as "award_types", "pi_names", "appl_ids", etc. For more information on how to specify these 
         #criteria in the dictionary, please refer to the RePorter API documentation (https://api.reporter.nih.gov/?urls.primaryName=V2.0).
        }}

# Passing data and headers to make a request
# The return is stored in r, a request object
r = requests.post(url, data=json.dumps(data), headers=headers)
# Parsing request object 'r' to a python dictionary using .json()
r_json = r.json()    
tot_records = r_json['meta']['total'] #Pulling out 'total' under 'meta' to see how many records matches our criteria

print(f"Total number of records: {tot_records}")

Total number of records: 12608


To retrieve all 12608 records of NCI projects in 2021 using a for loop, we must first calculate the number of iterations needed. By dividing the total number of records by the API's return limit of 500 records per request, we find that we need 25 full iterations and one final iteration of 108 records.

In this demonstration, we save both the meta data and actual results of each iteration separately. After all iterations are complete, the 'results' list will contain 26 sublists, each with either 500 or 108 dictionaries of all results. The 'meta' list will consist of 26 elements, each containing the meta data of the corresponding request.


In [3]:
'''
Query the Reporter API

API documentation:
https://api.reporter.nih.gov/?urls.primaryName=V2.0
'''

url = "https://api.reporter.nih.gov/v2/projects/search"
headers = {'accept': 'application/json', 'Content-Type': 'application/json'}

'''
Fetch all results through one loop.
'''

meta = []
results = []

for i in range(26):
    
    data = {
         "criteria":
         {
             "fiscal_years":[2021],
               "agencies":["NCI"],
             },
             "include_fields": [
                "ApplId","SubprojectId","FiscalYear","Organization", "ProjectNum","OrgCountry",
                "ProjectNumSplit","ContactPiName", "AllText","FullStudySection",
                "ProjectStartDate","ProjectEndDate"
             ],
             "offset":500*i,
             "limit":500,
             "sort_field":"project_start_date",
             "sort_order":"desc"
        }


    r = requests.post(url, data=json.dumps(data), headers=headers)
    r_json = r.json()

    if r_json['results'] != []:
            
        meta.append(r_json['meta']) #storing the return to meta
        results.append(r_json['results'])#storing the return to results
        print(f'Call #{i}') # Printing out the process for verification
        print(f"# Number of records in this call: {len(r_json['results'])}")
    else:
            break

Call #0
# Number of records in this call: 500
Call #1
# Number of records in this call: 500
Call #2
# Number of records in this call: 500
Call #3
# Number of records in this call: 500
Call #4
# Number of records in this call: 500
Call #5
# Number of records in this call: 500
Call #6
# Number of records in this call: 500
Call #7
# Number of records in this call: 500
Call #8
# Number of records in this call: 500
Call #9
# Number of records in this call: 500
Call #10
# Number of records in this call: 500
Call #11
# Number of records in this call: 500
Call #12
# Number of records in this call: 500
Call #13
# Number of records in this call: 500
Call #14
# Number of records in this call: 500
Call #15
# Number of records in this call: 500
Call #16
# Number of records in this call: 500
Call #17
# Number of records in this call: 500
Call #18
# Number of records in this call: 500
Call #19
# Number of records in this call: 500
Call #20
# Number of records in this call: 500
Call #21
# Number of re

### Option 2: Using a while loop

To streamline the process of retrieving all records, we can use a while loop instead of a for loop. This eliminates the need to know beforehand the number of iterations required. 

The script below continuously sends requests and saves the returned results until all records are exhausted. The loop is broken when the request is not a dictionary and returns a list with a string message reading "Requested offset X exceeded total records count of Y". 

In [4]:
'''
Query the Reporter API

API documentation:
https://api.reporter.nih.gov/?urls.primaryName=V2.0
'''

url = "https://api.reporter.nih.gov/v2/projects/search"
headers = {'accept': 'application/json', 'Content-Type': 'application/json'}


'''
Fetch all results through while loop.
'''

meta = []
results = []
i = 0
r_json = {}

while True:
    
    data = {
         "criteria":
         {
             "fiscal_years":[2021],
               "agencies":["NCI"],
             },
             "include_fields": [
                "ApplId","SubprojectId","FiscalYear","Organization", "ProjectNum","OrgCountry",
                "ProjectNumSplit","ContactPiName", "AllText","FullStudySection",
                "ProjectStartDate","ProjectEndDate"
             ],
             "offset":500*i,
             "limit":500,
             "sort_field":"project_start_date",
             "sort_order":"desc"
        }

    r = requests.post(url, data=json.dumps(data), headers=headers)
    r_json = r.json()
    
    
    if type(r_json) == dict:
     
        meta.append(r_json['meta'])
        results.append(r_json['results'])
        print(f'Call #{i}')
        print(f"# Number of records in this call: {len(r_json['results'])}")
        i+=1
    else:
        break


print(f'Number of retrievals done: {len(results)}')
total = 0
for i in range(len(results)):
    total += len(results[i])

print(f"All matches are returned. Total records: {total}")

            
            
    

Call #0
# Number of records in this call: 500
Call #1
# Number of records in this call: 500
Call #2
# Number of records in this call: 500
Call #3
# Number of records in this call: 500
Call #4
# Number of records in this call: 500
Call #5
# Number of records in this call: 500
Call #6
# Number of records in this call: 500
Call #7
# Number of records in this call: 500
Call #8
# Number of records in this call: 500
Call #9
# Number of records in this call: 500
Call #10
# Number of records in this call: 500
Call #11
# Number of records in this call: 500
Call #12
# Number of records in this call: 500
Call #13
# Number of records in this call: 500
Call #14
# Number of records in this call: 500
Call #15
# Number of records in this call: 500
Call #16
# Number of records in this call: 500
Call #17
# Number of records in this call: 500
Call #18
# Number of records in this call: 500
Call #19
# Number of records in this call: 500
Call #20
# Number of records in this call: 500
Call #21
# Number of re

### Saving all the retrieved records

Both options 1 and 2 above store the iteration returns in separate lists of 'meta' and 'results'. However, this is not the only method to save the returns. Another approach is to save the returned dictionaries from each request as pandas dataframes and concactenate them as the iteration moves forward.

The following code blocks demonstrate how to output the lists of returns obtained using either option 1 or 2. A simple function named 'results_2_csv' is demonstrated. 

In [5]:
import itertools
import pandas as pd

def result_list_2_csv(input_list, file_name, meta):
    '''
    Transform the retrieved records to csv files.
    For meta data, each row in the resulting csv represents the meta data of each retrieval
    For results, each row in the resulting csv represents each record
    
    Input:
        input_list: results or meta data retreived
        file_name: str, file name wished to save under
        meta: bool, true if transforming meta data
    '''
    if not meta:
        # flatten records from each retrieval to the same level
        all_records = list(itertools.chain(*input_list))
    else:
        all_records = input_list
    
    #unpack nested dictionaries to same-leveled columns
    all_records_df = pd.json_normalize(all_records) 
    all_records_df.to_csv(file_name)

In [6]:
# Call result_list_2_csv
result_list_2_csv(results, 'results.csv', meta = False) #generate a csv for all records
result_list_2_csv(meta, 'meta.csv', meta = True)  #generate a csv for all meta data of the retrievals

### Conclusion

This tutorial has demonstrated how to effectively retrieve all records that match specified criteria through the RePorter API. We have shown two different methods of accomplishing this: using a for loop with a known number of iterations, or using a more flexible and fail-safe while loop. The returns of each iteration in this tutorial are saved in separate lists, while other approaches are available. With the ability to specify criteria such as agency, fiscal year, award type, PI names and more, users can easily extract the data they need from the API and analyze it in a manner that suits their needs.